# Running a fixed-length TPS simulation

This is file runs the main calculation for the fixed length TPS simulation. It requires the file `alanine_dipeptide_fixed_tps_traj.nc`, which is written in the notebook `alanine_dipeptide_fixed_tps_traj.ipynb`.

In this file, you will learn:
* how to set up and run a fixed length TPS simulation

NB: This is a long calculation. In practice, it would be best to export the Python from this notebook, remove the `live_visualizer`, and run non-interactively on a computing node.

In [1]:
from __future__ import print_function
import openpathsampling as paths

## Load engine, trajectory, and states from file

In [2]:
old_storage = paths.Storage("ad_fixed_tps_traj.nc", "r")

In [3]:
engine = old_storage.engines['300K']
network = old_storage.networks['fixed_tps_network']
traj = old_storage.trajectories[0]

## TPS

The only difference between this and the flexible path length example in `alanine_dipeptide_tps_run.ipynb` is that we used a `FixedLengthTPSNetwork`. We selected the `length=400` (8 ps) as a maximum length based on the results from a flexible path length run.

In [4]:
scheme = paths.OneWayShootingMoveScheme(network,
                                        selector=paths.UniformSelector(),
                                        engine=engine)

In [5]:
initial_conditions = scheme.initial_conditions_from_trajectories(traj)

No missing ensembles.
No extra ensembles.


In [6]:
storage = paths.Storage("ad_fixed_tps.nc", "w")
storage.save(initial_conditions);  # save these to give storage a template

In [7]:
sampler = paths.PathSampling(storage=storage,
                             move_scheme=scheme,
                             sample_set=initial_conditions)

In [8]:
sampler.run(10000)

Working on Monte Carlo cycle number 10000
Running for 10 hours 43 minutes 11 seconds -  3.86 seconds per step
Estimated time remaining: 3 seconds
DONE! Completed 10000 Monte Carlo cycles.


In [9]:
old_storage.close()
storage.close()

With this done, you can go on to do the fixed-length parts of the analysis in `alanine_dipeptide_tps_analysis.ipynb`.